<a href="https://colab.research.google.com/github/ronron-gh/ESP32_EdgeAI/blob/main/nnabla_convert_mydata_model_to_csrc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

nnablaをインストールする。

In [1]:
!pip install nnabla-ext-cuda114

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.2/167.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstallin

numpyを1.23.0に下げて、pillowを9.1.0に上げる（そうしないと、この後のclassification.pyの実行でエラーになった。数カ月前まではnumpyの変更だけで動いていたので、今後もこのようなバージョンの不整合は発生するかもしれない）。

In [2]:
!pip install numpy==1.23.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


In [3]:
!pip install pillow==9.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 38.4 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


nnablaのサンプル群をGitHubから取得。

In [4]:
!git clone https://github.com/sony/nnabla-examples.git

Cloning into 'nnabla-examples'...
remote: Enumerating objects: 9292, done.
remote: Counting objects: 100% (1850/1850), done.
remote: Compressing objects: 100% (722/722), done.
remote: Total 9292 (delta 1090), reused 1769 (delta 1077), pack-reused 7442
Receiving objects: 100% (9292/9292), 296.48 MiB | 35.13 MiB/s, done.
Resolving deltas: 100% (4992/4992), done.
Updating files: 100% (1676/1676), done.


本ブログで公開しているデータセットやプログラムをGitHubから取得。

In [5]:
!git clone https://github.com/ronron-gh/ESP32_EdgeAI.git

Cloning into 'ESP32_EdgeAI'...
remote: Enumerating objects: 591, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (514/514), done.
remote: Total 591 (delta 81), reused 575 (delta 71), pack-reused 0
Receiving objects: 100% (591/591), 4.80 MiB | 38.38 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [6]:
%cd ESP32_EdgeAI

/content/ESP32_EdgeAI


データセットをnnablaに読み込ませるためのリストファイル(train.csv, test.csv)を作成する。画像は28x28のモノクロに変換する。（CSVの作成と画像の変換を行うPythonスクリプトを用意しました。）

In [7]:
!python make_dataset_csv.py my_dataset

The current folder is my_dataset
The current folder is my_dataset/2
listed file : my_dataset/2/0110.jpg
listed file : my_dataset/2/0087.jpg
listed file : my_dataset/2/0103.jpg
listed file : my_dataset/2/0085.jpg
listed file : my_dataset/2/0113.jpg
listed file : my_dataset/2/0095.jpg
listed file : my_dataset/2/0082.jpg
listed file : my_dataset/2/0123.jpg
listed file : my_dataset/2/0107.jpg
listed file : my_dataset/2/0096.jpg
listed file : my_dataset/2/0108.jpg
listed file : my_dataset/2/0097.jpg
listed file : my_dataset/2/0125.jpg
listed file : my_dataset/2/0116.jpg
listed file : my_dataset/2/0086.jpg
listed file : my_dataset/2/0128.jpg
listed file : my_dataset/2/0081.jpg
listed file : my_dataset/2/0126.jpg
listed file : my_dataset/2/0102.jpg
listed file : my_dataset/2/0119.jpg
listed file : my_dataset/2/0197.jpg
listed file : my_dataset/2/0100.jpg
listed file : my_dataset/2/0114.jpg
listed file : my_dataset/2/0088.jpg
listed file : my_dataset/2/0112.jpg
listed file : my_dataset/2/0195.

train.csv, test.csv, converted_datasetsフォルダが作成されていることを確認する。

In [8]:
!ls

Arduino		     my_dataset				       README.md
converted_datasets   nnabla_convert_mnist_model_to_csrc.ipynb  test.csv
make_dataset_csv.py  nnabla-example-modify		       train.csv


nnablaのMNISTサンプルコードのディレクトリに、作成したCSV、データセット（変換後）、改造したプログラムをコピー。

In [9]:
!cp *.csv ../nnabla-examples/image-classification/mnist-collection/

In [10]:
!cp -r converted_datasets ../nnabla-examples/image-classification/mnist-collection/

In [11]:
!cp nnabla-example-modify/classification_mydata.py ../nnabla-examples/image-classification/mnist-collection/

mnistサンプルプログラムのフォルダに移動し、学習を実行。

In [12]:
%cd ../nnabla-examples/image-classification/mnist-collection/

/content/nnabla-examples/image-classification/mnist-collection


In [13]:
!python classification_mydata.py -c cudnn -n lenet -o output

2023-02-12 14:07:16,368 [nnabla][INFO]: Initializing CPU extension...
2023-02-12 14:07:17,712 [nnabla][INFO]: Running in cudnn
2023-02-12 14:07:17,937 [nnabla][INFO]: Initializing CUDA extension...
2023-02-12 14:07:18,053 [nnabla][INFO]: Initializing cuDNN extension...
2023-02-12 14:07:23,530 [nnabla][INFO]: Saving output/lenet_result_epoch0.nnp as nnp
2023-02-12 14:07:23,530 [nnabla][INFO]: Saving <_io.StringIO object at 0x7f36f213edc0> as prototxt
2023-02-12 14:07:23,545 [nnabla][INFO]: Parameter save (.h5): <_io.BytesIO object at 0x7f36f20f2e50>
2023-02-12 14:07:23,546 [nnabla][INFO]: Model file is saved as (.nnp): output/lenet_result_epoch0.nnp
2023-02-12 14:07:23,546 [nnabla][INFO]: DataSource with shuffle(True)
2023-02-12 14:07:23,549 [nnabla][INFO]: Using DataSourceWithFileCache
2023-02-12 14:07:23,549 [nnabla][INFO]: DataSource with shuffle(True)
2023-02-12 14:07:23,549 [nnabla][INFO]: Cache Directory is None
2023-02-12 14:07:23,549 [nnabla][INFO]: Cache size is 100
2023-02-12 

outputフォルダが作られていることを確認。

In [14]:
!ls

args.py			  converted_datasets  README.md		train.csv
_checkpoint_nnp_util.py   dcgan.py	      requirements.txt	vae.py
classification_bnn.py	  mnist_data.py       siamese.py	vat.py
classification_mydata.py  output	      test.csv
classification.py	  __pycache__	      tmp.monitor


Cソースコードの出力先フォルダを作成。

In [15]:
!mkdir ./output_csrc

学習済みモデルファイル(.nnp)をCソースコードに変換。

In [16]:
!nnabla_cli convert -O CSRC -b 1 ./output/lenet_result.nnp ./output_csrc

2023-02-12 14:08:28,118 [nnabla][INFO]: Initializing CPU extension...
NNabla command line interface (Version:1.33.1, Build:230206062328)
2023-02-12 14:08:29,198 [nnabla][WARNING]: The export file format is 'CSRC' or 'SAVED_MODEL' that argument '--export-format' will have to be set!!!
Importing ./output/lenet_result.nnp
 Expanding Validation.
Using network [Validation].


変換の結果、次の４つのファイルが出来ていることを確認（その他のファイルは使わない）。
Validation_inference.c
Validation_inference.h
Validation_parameters.c
Validation_parameters.h

In [17]:
!ls -all output_csrc

total 568
drwxr-xr-x 2 root root   4096 Feb 12 14:08 .
drwxr-xr-x 7 root root   4096 Feb 12 14:08 ..
-rw-r--r-- 1 root root    902 Feb 12 14:08 GNUmakefile
-rw-r--r-- 1 root root   2124 Feb 12 14:08 Validation_example.c
-rw-r--r-- 1 root root  23761 Feb 12 14:08 Validation_inference.c
-rw-r--r-- 1 root root   2417 Feb 12 14:08 Validation_inference.h
-rw-r--r-- 1 root root 529580 Feb 12 14:08 Validation_parameters.c
-rw-r--r-- 1 root root    983 Feb 12 14:08 Validation_parameters.h


変換後のファイルをPCにダウンロードするために、Google Driveに一旦コピーする。

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
!cp -r output_csrc /content/drive/MyDrive/